# Symforce

## Introduction

[SymForce](https://symforce.org/) is a fast symbolic computation and code generation library for robotics applications like computer vision, state estimation, motion planning, and controls. It combines the development speed and flexibility of symbolic mathematics with the performance of autogenerated, highly optimized code in C++ or any target runtime language. SymForce contains three independently useful systems:

**Symbolic Toolkit** - builds on the SymPy API to provide rigorous geometric and camera types, lie group calculus, singularity handling, and tools to model complex problems

**Code Generator** - transforms symbolic expressions into blazing-fast, branchless code with clean APIs and minimal dependencies, with a template system to target any language

**Optimization Library** - a fast tangent-space optimization library based on factor graphs, with a highly optimized implementation for real-time robotics applications

SymForce automatically computes tangent space Jacobians, eliminating the need for any bug-prone handwritten derivatives. Generated functions can be directly used as factors in our nonlinear optimizer. This workflow enables faster runtime functions, faster development time, and fewer lines of handwritten code versus alternative methods.

SymForce is developed and maintained by **Skydio**. It is used in production to accelerate tasks like SLAM, bundle adjustment, calibration, and sparse nonlinear MPC for autonomous robots at scale.

## Tutorials

### symforce.symbolic.Pose3 and  sym.Pose3

SymForce provides **sym** packages with **runtime code** for geometry and camera types, e.g. `sym.Pose3` that are generated from its **symbolic geo and cam packages**, e.g. `symforce.symbolic.Pose3`. As such, there are multiple versions of a class like Pose3 and it can be a common source of confusion.

In [19]:
import symforce.symbolic as sf
q = sf.Quaternion(xyz = sf.V3(0.0, 0.0, 0.0), w=sf.Scalar(1.0))
R = sf.Rot3(q)
t = sf.V3(0.0, 0.0, 0.0)
sf.Pose3(R, t)

<Pose3 R=<Rot3 <Q xyzw=[0.0, 0.0, 0.0, 1.0]>>, t=(0.0, 0.0, 0.0)>

In [44]:
import symforce.symbolic as sf
import sym
print(f"sym package rotation matrix type: {type(sym.Pose3.identity().R.to_rotation_matrix())}")
print(f"symbolic package rotation matrix type: {type(sf.Pose3.identity().R.to_rotation_matrix())}")
print(f"symbolic package rotation matrix to_numpy type: { type(sf.Pose3.identity().R.to_rotation_matrix().to_numpy())}")


sym package rotation matrix type: <class 'numpy.ndarray'>
symbolic package rotation matrix type: <class 'symforce.geo.matrix.Matrix33'>
symbolic package rotation matrix to_numpy type: <class 'numpy.ndarray'>


In [30]:
# k3d plot helper

import k3d
import math
import numpy as np
import time

# TODO(xipeng.wang): Update this to use symforce directly instead of numpy
class XYZ_RBG_Axes():
    def __init__(self, plot, name, R, t, color=0xffff00, axis_length = 1):
        self.axis_length = axis_length
        self.plot = plot
        self.R = R
        self.t = t.reshape(3, 1)
        self.name = name
        origin = self.t
        x_axis = np.stack((origin, origin + self.R.dot(np.array([axis_length, 0., 0.])).reshape(3,1)), axis=0)
        y_axis = np.stack((origin, origin + self.R.dot(np.array([0., axis_length, 0.])).reshape(3,1)), axis=0)
        z_axis = np.stack((origin, origin + self.R.dot(np.array([0., 0., axis_length])).reshape(3,1)), axis=0)
        self.xline_plot = k3d.line(x_axis, shader='mesh', width=0.05, color=0xff0000, name=name+'_xaxis')
        self.yline_plot = k3d.line(y_axis, shader='mesh', width=0.05, color=0x0000ff, name=name+'_yaxis')
        self.zline_plot = k3d.line(z_axis, shader='mesh', width=0.05, color=0x00ff00, name=name+'_zaxis')
        self.xpoint = k3d.points(origin + self.R.dot(np.array([axis_length, 0., 0.])).reshape(3,1), point_size=0.3, 
                                  shader='3d', color=color, name=self.name+'_end_point_x')
        self.ypoint = k3d.points(origin + self.R.dot(np.array([0., axis_length, 0.])).reshape(3,1), point_size=0.3, 
                                  shader='3d', color=color, name=self.name+'_end_point_y')
        self.zpoint = k3d.points(origin + self.R.dot(np.array([0., 0., axis_length])).reshape(3,1), point_size=0.3, 
                                  shader='3d', color=color, name=self.name+'_end_point_z')
        self.plot += self.xline_plot
        self.plot += self.yline_plot
        self.plot += self.zline_plot
        self.plot += self.xpoint
        self.plot += self.ypoint
        self.plot += self.zpoint 
    def update_pose(self, R, t):
        self.R = R
        self.t = t.reshape(3,1)
        origin = self.t
        axis_length = self.axis_length
        x_axis = np.stack((origin, origin + self.R.dot(np.array([axis_length, 0., 0.])).reshape(3,1)), axis=0)
        y_axis = np.stack((origin, origin + self.R.dot(np.array([0., axis_length, 0.])).reshape(3,1)), axis=0)
        z_axis = np.stack((origin, origin + self.R.dot(np.array([0., 0., axis_length])).reshape(3,1)), axis=0)
        self.xline_plot.vertices = x_axis
        self.xpoint.positions = origin + self.R.dot(np.array([axis_length, 0., 0.])).reshape(3,1)
        self.yline_plot.vertices = y_axis
        self.ypoint.positions = origin + self.R.dot(np.array([0., axis_length, 0.])).reshape(3,1)
        self.zline_plot.vertices = z_axis
        self.zpoint.positions = origin + self.R.dot(np.array([0., 0., axis_length])).reshape(3,1)


In [ ]:
# k3d plot example

plot = k3d.plot(camera_auto_fit=True, grid=(0, 0, 0, 5, 5, 5))

R = sf.Rot3().to_rotation_matrix().to_numpy()
t = sf.V3(0, 0, 0).to_numpy()
C0 = XYZ_RBG_Axes(plot, 'C0', R, t)

R = sf.Rot3.random().to_rotation_matrix().to_numpy()
t = sf.V3(1, 0, 0).to_numpy()
C1 = XYZ_RBG_Axes(plot, 'C1', R, t)

plot.display()

## Solve Problems in Real World